Aproximate Nearest Neighbors search using Indexes in search. 

Following are the ANN algorithms

1. Exhaustive search
2. LSH
3. Product Quantization
4. Tree and Graph
5. HNSW

Dataset used to implement the above ANN algorithms is sift 1M. This dataset is used to evaluate the quality of ANN search algorithms.

Tha dataset consists of three types of vectors
1. base vectors
2. query vectors
3. learning vectors.
the vector files are stored in .bvencs or .fvecs format. These all files are compressed in a .tar format.

In [3]:
import shutil
import urllib.request as request
from contextlib import closing
import tarfile
import numpy as np

Download the dataset

In [4]:
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

once .tar file is downloaded, extract all the files (untar it)

In [5]:
tar_file = tarfile.open('sift.tar.gz', "r:gz")
tar_file.extractall()

the vector files extracted are in .fvecs format. Read these files and store th vector.

Vectors consists of integer values with 128 dimenional vector. 

In [6]:
vector_filepath = "/content/sift/sift_base.fvecs"
read_file = np.fromfile(vector_filepath, dtype='int32')
dimension = read_file[0]
print("base vector dimension: ", dimension)
base_vectors = read_file.reshape(-1, dimension+1)[:, 1:].copy().view('float32')

base vector dimension:  128


Read query vector from the extracted files. The query vector is of format .fvecs

In [7]:
queryVector_filepath = "/content/sift/sift_query.fvecs"
read_file = np.fromfile(queryVector_filepath, dtype='int32')
dimension = read_file[0]
print("query vector dimension: ", dimension)
query_vector = read_file.reshape(-1, dimension+1)[:, 1:].copy().view('float32')
query_vector = query_vector[0].reshape(1, query_vector.shape[1])

query vector dimension:  128


Installing faiss.

faiss(Facebook AI Similarity Search) is one of the Index solution used for searching nearest neighbors.

Vectors are stored in faiss and the new index vector is used to query the query vector to compared with other index vector to find the nearest neighbors. 

In [8]:
pip install faiss-cpu

     |████████████████████████████████| 8.4 MB 20.7 MB/s 


1. EXHAUSTIVE SEARCH

It uses "INDEXFLATL2" from faiss libbrary.

Exhaustive search has high search-quality but it takes long time to search. These are Flat indexes as we do not change/modify the vectors stored. As there is no approximation or clustering of the vectors from the dataset, indexes generated with more accurate results.

In [9]:
import faiss
from time import time

start_time = time()
vectorindex = faiss.IndexFlatL2(base_vectors.shape[1])
vectorindex.add(base_vectors)
distances, indices = vectorindex.search(query_vector, 10) # finding the indices and distance, selecting the top 10 nearest neigbors.
end_time = time()
print("time for exhastive search: %.2fs" %(end_time-start_time))

time for exhastive search: 0.26s


In [10]:
print("top 10 nearest neighbors", indices)
print("Their disnatce to query vector", distances)

top 10 nearest neighbors [[932085 934876 561813 708177 706771 695756 435345 701258 455537 872728]]
Their disnatce to query vector [[54229. 55091. 59531. 65260. 65697. 67010. 68247. 69844. 71441. 71861.]]


2. Locality Sensitive Hashing:

LSH is one of the vector encoding technique. To construct the index, map the data points to the buckets. The data points near to each other are located in the same bucket.

To search for the constructed index, the query point is hashed to get the set of nearest data points i.e, the nearest bucket.

As we construct the new index, number of bits in index can b specified while constructing it.

LSH performance is dependent on the parameters set and it has poor performance for high-dimensional data.

In [11]:
start_time = time()

no_bits = base_vectors.shape[1]*4  # resolution of bucketed vector
vectorindex = faiss.IndexLSH(base_vectors.shape[1], no_bits) #space complexity used to store indix is reduced
vectorindex.add(base_vectors)
distances, indices = vectorindex.search(query_vector, 10) # finding the indices and distance, selecting the top k nearest neigbors.

end_time = time()
print("time for LSH: %.2fs" %(end_time-start_time))

time for LSH: 3.97s


In [12]:
print("top 10 nearest neighbors", indices)
print("Their disnatce to query vector", distances)

top 10 nearest neighbors [[435345 931632 813701 934876 708177 455537 932085 561813 248185 361496]]
Their disnatce to query vector [[75. 75. 76. 76. 76. 77. 77. 78. 79. 81.]]


3. PRODUCT QUANTIZATION

Product Quantization is not the optiomal solution to find the neaest neighbors. PQ combined with IVF steps to improve search speed. 

In [13]:
start_time = time()

dimension = base_vectors.shape[1]
subvector_size = 8
number_of_partition = 1024  # partitions must be >= pow(2, subvector_size)
search_in_x_partitions = 2 
quantizer = faiss.IndexFlatL2(dimension)
vectorindex = faiss.IndexIVFPQ(quantizer, dimension, number_of_partition, search_in_x_partitions, subvector_size)
vectorindex.train(base_vectors)
vectorindex.add(base_vectors)
distances, indices = vectorindex.search(query_vector, 10) # finding the indices and distance, selecting the top k nearest neigbors.

end_time = time()
print("time for PQ: %.2fs" %(end_time-start_time))

time for PQ: 30.04s


In [14]:
print("top 10 nearest neighbors", indices)
print("Their disnatce to query vector", distances)

top 10 nearest neighbors [[529986 727687 785802 259327 974147 325658 322550 455537 670103 170996]]
Their disnatce to query vector [[48216.6   50419.93  50419.93  50419.93  51095.47  51128.23  54210.59
  54312.965 55429.23  56660.438]]


4. Hierarchical Navigable Small World Graphs

HSNW tops in the ANN search algorithms with high performance. It has a graph structure, vertices are connected by edges to their nearest neighbors these are NSW graphs. Facebook is an example for NSW graphs. 
HNSW is a algorithm with great search quality and with good search time but still having the considerable index sizes.

In [16]:
start_time = time()

no_connections = 64  # number of connections each vertex will have
dimension = base_vectors.shape[1]
indvectorindexex = faiss.IndexHNSWFlat(dimension, no_connections)
vectorindex.add(base_vectors)
distances, indices = vectorindex.search(query_vector, 10)
end_time = time()
print("time for HNSW: %.2fs" %(end_time-start_time))

time for HNSW: 8.87s


In [17]:
print("top 10 nearest neighbors", indices)
print("Their disnatce to query vector", distances)

top 10 nearest neighbors [[ 529986 1529986 2529986  727687 2259327  259327  785802 1727687 1785802
  1259327]]
Their disnatce to query vector [[48216.6  48216.6  48216.6  50419.93 50419.93 50419.93 50419.93 50419.93
  50419.93 50419.93]]


In [2]:
pip install nmslib

     |████████████████████████████████| 13.5 MB 32.8 MB/s 
     |████████████████████████████████| 188 kB 60.5 MB/s 


In [18]:
import nmslib
start_time = time()

dimension = base_vectors.shape[1]
vectorindex = nmslib.init(method='hnsw', space='cosinesimil')
vectorindex.addDataPointBatch(base_vectors)
vectorindex.createIndex({'post': 2})
distances, indices = vectorindex.knnQuery(query_vector, 10) # finding the indices and distance, selecting the top k nearest neigbors.
end_time = time()
print("time for HSNW: %.2fs" %(end_time-start_time))

time for HSNW: 1608.68s


In [19]:
print("top 10 nearest neighbors", indices)
print("Their disnatce to query vector", distances)

top 10 nearest neighbors [0.10511148 0.10670507 0.13518357 0.13841808 0.13885641 0.14197004
 0.14211208 0.14247584 0.14263022 0.14407969]
Their disnatce to query vector [932085 934876 701258 455537 872728  36538 562594 908244 600499 619660]


5. Tree and Graph Algo

This algo constructs many trees. Each tree is constructed using a random  set of split and search al the trees at a time for query vector using priority queue. 

This search algorithms works in Annoy. Tis algorithm is not the best at accuracy at its search for indices but it has many perofrmace gains. 

In [14]:
pip install annoy

     |████████████████████████████████| 646 kB 5.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391689 sha256=b6aa17c36bbe5dffc6797b764b310206c642d635ed12a21f43918e0909db78a2
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [63]:
import annoy
start_time = time()


dimension = 128
number_of_trees =5
vectorindex= annoy.AnnoyIndex(dimension)
for i, vec in enumerate(base_vectors):
  vectorindex.add_item(i, vec.tolist())
# vectorindex.add_item(base_vectors)
vectorindex.build(number_of_trees)
indices = vectorindex.get_nns_by_vector(query_vector[0], 10)
end_time = time()
print("time for Trees Annoy: %.2fs" %(end_time-start_time))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  import sys


time for Trees Annoy: 18.14s


In [64]:
print("top 10 nearest neighbors", indices)

top 10 nearest neighbors [934876, 562594, 908244, 565419, 746931, 619829, 544275, 104122, 806773, 883849]
